In [0]:
from pyspark import SparkConf,SparkContext 
from pyspark.sql import SparkSession


In [0]:
sc

<SparkContext master=spark://10.139.64.4:7077 appName=Databricks Shell>

In [0]:
spark

In [0]:
greentaxi_df=spark.read.csv("/Volumes/azuredatabricks_2000/default/greentaxi/2021_Green_Taxi_Trip_Data.csv",inferSchema=True,header=True)

In [0]:
greentaxi_df_final=greentaxi_df.repartition(10)

In [0]:
# greentaxi_df_final.write.mode("overwrite").option("header","true").parquet("/Volumes/azuredatabricks_2000/default/greentaxi/greenparquet")

In [0]:
schema=greentaxi_df_final.schema

In [0]:
taxiz_df=spark.readStream.schema(schema=schema).format("csv").option("header","true").load("/Volumes/azuredatabricks_2000/default/greentaxi/*parquet")

In [0]:
from pyspark.sql.functions import * 

tx_transformation = taxiz_df\
    .withColumn('timestamp', current_timestamp())\
    .groupBy("payment_type").sum("fare_amount").alias("fare_amounts")

df2 = df_stream \
              .withColumn('timestamp', unix_timestamp(col('EventDate'), "MM/dd/yyyy hh:mm:ss aa").cast(TimestampType())) \
              .withWatermark("timestamp", "1 minutes") \
              .groupBy(col("SendID"), "timestamp") \
              .agg(max(col('timestamp')).alias("timestamp")) \
              .orderBy('timestamp', ascending=False)

In [0]:
greentaxi_df.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [0]:
ta

DataFrame[payment_type: int, sum(fare_amount): double, timestamp: timestamp]

In [0]:
from pyspark.sql.functions import current_timestamp, window


taxiz_df_final = taxiz_df.withColumn("timestamp", current_timestamp())
tx_transformation_with_watermark = taxiz_df_final \
    .withWatermark("timestamp", "10 minutes")  
    
aggregated_df = tx_transformation_with_watermark \
    .groupBy(
        window(tx_transformation_with_watermark.timestamp, "5 minutes"),  
        "payment_type"
    ) \
    .sum("fare_amount")


tx_stream = aggregated_df \
    .writeStream \
    .outputMode("append") \
    .format("parquet") \
    .option("path", "/Volumes/azuredatabricks_2000/default/writestream") \
    .option("checkpointLocation", "/Volumes/azuredatabricks_2000/default/checkpoint/") \
    .start()

tx_stream.awaitTermination()

---------------------------------------------------------------------------
StreamingQueryException                   Traceback (most recent call last)
File <command-1194792206002261>, line 24
      8 aggregated_df = tx_transformation_with_watermark \
      9     .groupBy(
     10         window(tx_transformation_with_watermark.timestamp, "5 minutes"),  
     11         "payment_type"
     12     ) \
     13     .sum("fare_amount")
     16 tx_stream = aggregated_df \
     17     .writeStream \
     18     .outputMode("append") \
   (...)
     21     .option("checkpointLocation", "/Volumes/azuredatabricks_2000/default/checkpoint/") \
     22     .start()
---> 24 tx_stream.awaitTermination()

File /databricks/spark/python/pyspark/sql/streaming/query.py:221, in StreamingQuery.awaitTermination(self, timeout)
    219     return self._jsq.awaitTermination(int(timeout * 1000))
    220 else:
--> 221     return self._jsq.awaitTermination()

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zi

In [0]:
taxiz_df.stop()

---------------------------------------------------------------------------
PySparkAttributeError                     Traceback (most recent call last)
File <command-1194792206002259>, line 1
----> 1 taxiz_df.stop()

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/dataframe.py:3732, in DataFrame.__getattr__(self, name)
   3699 """Returns the :class:`Column` denoted by ``name``.
   3700 
   3701 .. versionadded:: 1.3.0
   (...)
   3729 +---+
   3730 """
   3731 if name not in self.columns:
-> 3732     raise PySparkAttributeError(
   3733         error_class="ATTRIBUTE_NOT_SUPPORTED", message_parameters={"attr_name": name}
   3